In [26]:
import httpx
from parsel import Selector

response = httpx.get(
    "https://www.glassdoor.com.mx/Descripci%C3%B3n-general/Trabajar-en-eBay-EI_IE7853.12,16.htm"
)
selector = Selector(response.text)
# find description in the HTML:
print(selector.css('[data-test="employerDescription"]::text').get())
# will print:
# eBay is where the world goes to shop, sell, and give. Every day, our professionals connect millions of buyers and sellers around the globe, empowering people and creating opportunity. We're on a mission to build a better, more connected form of commerce that benefits individuals

None


In [24]:
import re
import json
import httpx

response = httpx.get(
    "https://www.glassdoor.com/",
    follow_redirects=True,
)

country_matches = re.findall(r'"countryMenu\\":.+?(\[.+?\])', response.text)
if country_matches:
    country_data = country_matches[0].replace('\\', '')
    country_data = json.loads(country_data)
    for country in country_data:
        print(f"{country['textKey']}: {country['id']}")
else:
    print("Country data not found on the page.")

Country data not found on the page.


In [22]:
from parsel import Selector
import httpx

france_location_cookie = {"tldp": "12"}
response = httpx.get(
    "https://www.glassdoor.com/Overview/Working-at-eBay-EI_IE7853.11,15.htm",
    cookies=france_location_cookie,
    follow_redirects=True,
)
selector = Selector(response.text)
# find employee count in the HTML:
print(selector.css('[data-test="employer-size"]::text').get())
# will print:
# Plus de 10 000 employés

None


In [16]:
function addGlobalStyle(css) {
    var head, style;
    head = document.getElementsByTagName('head')[0];
    if (!head) { return; }
    style = document.createElement('style');
    style.type = 'text/css';
    style.innerHTML = css;
    head.appendChild(style);
}

addGlobalStyle("#HardsellOverlay {display:none !important;}");
addGlobalStyle("body {overflow:auto !important; position: initial !important}");

window.addEventListener("scroll", event => event.stopPropagation(), true);
window.addEventListener("mousemove", event => event.stopPropagation(), true);

SyntaxError: invalid syntax (108858732.py, line 1)

In [14]:
import json
import httpx


def find_companies(query: str):
    """find company Glassdoor ID and name by query. e.g. "ebay" will return "eBay" with ID 7853"""
    result = httpx.get(
        url=f"https://www.glassdoor.com/searchsuggest/typeahead?numSuggestions=8&source=GD_V2&version=NEW&rf=full&fallback=token&input={query}",
    )
    data = json.loads(result.content)
    return data[0]["suggestion"], data[0]["employerId"]

print(find_companies("ebay"))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)